![Logo](https://www.up.edu.mx/wp-content/themes/panamericana/img/logo-footer.png)

# **Tarea 6: Regresión Lineal Múltiple**

## **Asignatura:** Econometría  
## **Profesora:** Mae. Julieta María Sarralde Vera

---

**Nombre del Alumno:**  
Luis Enrique Noguera Gil

**Fecha de entrega:**  
08 de septiembre de 2024


# Auto reload modules

In [8]:
%load_ext autoreload
%autoreload 2

In [9]:
import scripts.index as index
from scipy.stats import t
from scipy import stats
import fs

In [10]:
MAIN_DIR = fs.open_fs("../../data/raw/HW06")
T7_6_DIR = MAIN_DIR.getsyspath("Table 7_6.xls")
T7_8_DIR = MAIN_DIR.getsyspath("Table 7_8.xls")
T7_12_DIR = MAIN_DIR.getsyspath("Table 7_12.xls")

In [11]:
DIR = fs.open_fs(".")
NB_DIR = DIR.getsyspath("0.0-leng-HW06.ipynb")

# Extrae los encabezados y genera el índice
headings = index.extract_headings(NB_DIR)
toc = index.generate_toc(headings)

# Muestra el índice generado
# print(toc)

- [**Tarea 6: Regresión Lineal Múltiple**](#**tarea-6:-regresión-lineal-múltiple**)
  - [**Asignatura:** Econometría](#**asignatura:**-econometría)
  - [**Profesora:** Mae. Julieta María Sarralde Vera](#**profesora:**-mae.-julieta-maría-sarralde-vera)
- [Auto reload modules](#auto-reload-modules)
- [1. Estimación de Regresiones](#1.-estimación-de-regresiones)
    - [a) ¿Es $ \alpha_2 = \beta_2 $? ¿Por qué?](#a)-¿es-$-\alpha_2-=-\beta_2-$?-¿por-qué?)
    - [b) ¿Es $ \lambda_3 = \beta_3 $? ¿Por qué?](#b)-¿es-$-\lambda_3-=-\beta_3-$?-¿por-qué?)
- [2. La demanda de rosas](#2.-la-demanda-de-rosas)
    - [a) Estime los parámetros del modelo lineal e interprete los resultados.](#a)-estime-los-parámetros-del-modelo-lineal-e-interprete-los-resultados.)
    - [b) Estime los parámetros del modelo log-lineal e interprete los resultados.](#b)-estime-los-parámetros-del-modelo-log-lineal-e-interprete-los-resultados.)
    - [c) $ \beta_2 $, $ \beta_3 $ y $ \beta_4 $ dan respectivamente las elasticidades de la demanda respecto del precio propio, precio cruzado e ingreso. ¿Cuáles son, a priori, los signos de estas elasticidades? ¿Concuerdan estos resultados con las expectativas a priori?](#c)-$-\beta_2-$,-$-\beta_3-$-y-$-\beta_4-$-dan-respectivamente-las-elasticidades-de-la-demanda-respecto-del-precio-propio,-precio-cruzado-e-ingreso.-¿cuáles-son,-a-priori,-los-signos-de-estas-elasticidades?-¿concuerdan-estos-resultados-con-las-expectativas-a-priori?)
    - [d) ¿Cómo calcularía las elasticidades precio propio, precio cruzado e ingreso en el modelo lineal?](#d)-¿cómo-calcularía-las-elasticidades-precio-propio,-precio-cruzado-e-ingreso-en-el-modelo-lineal?)
    - [e) Con base en el análisis, ¿cuál modelo, si existe, escogería y por qué?](#e)-con-base-en-el-análisis,-¿cuál-modelo,-si-existe,-escogería-y-por-qué?)
- [3. Desembolsos del presupuesto de defensa de Estados Unidos, 1962-1981](#3.-desembolsos-del-presupuesto-de-defensa-de-estados-unidos,-1962-1981)
    - [a) Estime los parámetros de este modelo y sus errores estándar, y obtenga $ R^2 $ y $ R^2 $ ajustada.](#a)-estime-los-parámetros-de-este-modelo-y-sus-errores-estándar,-y-obtenga-$-r^2-$-y-$-r^2-$-ajustada.)
    - [b) Comente los resultados, considerando cualquier expectativa a priori que tenga sobre la relación entre $ Y $ y las diversas variables $ X $.](#b)-comente-los-resultados,-considerando-cualquier-expectativa-a-priori-que-tenga-sobre-la-relación-entre-$-y-$-y-las-diversas-variables-$-x-$.)
    - [c) ¿Qué otra(s) variable(s) incluiría en el modelo y por qué?](#c)-¿qué-otra(s)-variable(s)-incluiría-en-el-modelo-y-por-qué?)
- [4. Gasto de consumo real, ingreso real, riqueza real y tasas de interés reales de Estados Unidos de 1947 a 2000.](#4.-gasto-de-consumo-real,-ingreso-real,-riqueza-real-y-tasas-de-interés-reales-de-estados-unidos-de-1947-a-2000.)
    - [a) Con la información de la tabla, estime la función de consumo lineal usando los datos de ingreso, riqueza y tasa de interés. ¿Cuál es la ecuación ajustada?](#a)-con-la-información-de-la-tabla,-estime-la-función-de-consumo-lineal-usando-los-datos-de-ingreso,-riqueza-y-tasa-de-interés.-¿cuál-es-la-ecuación-ajustada?)
    - [b) ¿Qué indican los coeficientes estimados sobre las relaciones entre las variables y el gasto de consumo?](#b)-¿qué-indican-los-coeficientes-estimados-sobre-las-relaciones-entre-las-variables-y-el-gasto-de-consumo?)

In [12]:
# Librerías

import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")
sns.set_style("darkgrid")

# 1. Estimación de Regresiones

Considere los datos de la tabla:

| Y | X₂ | X₃ |
|---|----|----|
| 1 |  1 |  2 |
| 3 |  2 |  1 |
| 8 |  3 | -3 |

Con base en estos datos, estime las siguientes regresiones:

- $ Y_i = \alpha_1 + \alpha_2 X_{2i} + u_{1i} $ (1)
- $ Y_i = \lambda_1 + \lambda_3 X_{3i} + u_{2i} $ (2)
- $ Y_i = \beta_1 + \beta_2 X_{2i} + \beta_3 X_{3i} + u_i $ (3)

Nota: Estime sólo los coeficientes y no los errores estándar.

### a) ¿Es $ \alpha_2 = \beta_2 $? ¿Por qué?

### b) ¿Es $ \lambda_3 = \beta_3 $? ¿Por qué?

¿Qué conclusión importante obtiene de este ejercicio?

# 2. La demanda de rosas

En la tabla 7.6 (archivo anexo) se presenta la información trimestral sobre estas variables:

- $ Y = $ cantidad de rosas vendidas, docenas
- $ X_2 = $ precio promedio al mayoreo de las rosas, $/docena
- $ X_3 = $ precio promedio al mayoreo de los claveles, $/docena
- $ X_4 = $ ingreso familiar disponible promedio semanal, $/semana
- $ X_5 = $ variable de tendencia que toma valores de 1, 2, y así sucesivamente, durante el periodo 1971-III a 1975-II en el área metropolitana de Detroit.

Se le pide considerar las siguientes funciones de demanda:

- $ Y_t = \alpha_1 + \alpha_2 X_{2t} + \alpha_3 X_{3t} + \alpha_4 X_{4t} + \alpha_5 X_{5t} + u_t $
- $ \ln Y_t = \beta_1 + \beta_2 \ln X_{2t} + \beta_3 \ln X_{3t} + \beta_4 \ln X_{4t} + \beta_5 X_{5t} + u_t $

### a) Estime los parámetros del modelo lineal e interprete los resultados.

### b) Estime los parámetros del modelo log-lineal e interprete los resultados.

### c) $ \beta_2 $, $ \beta_3 $ y $ \beta_4 $ dan respectivamente las elasticidades de la demanda respecto del precio propio, precio cruzado e ingreso. ¿Cuáles son, a priori, los signos de estas elasticidades? ¿Concuerdan estos resultados con las expectativas a priori?

### d) ¿Cómo calcularía las elasticidades precio propio, precio cruzado e ingreso en el modelo lineal?

### e) Con base en el análisis, ¿cuál modelo, si existe, escogería y por qué?

**TABLA 7.6**  
Demanda trimestral de rosas en el área metropolitana de Detroit, de 1971-III a 1975-II

| Año y trimestre | Y (docenas vendidas) | X₂ ($/docena, rosas) | X₃ ($/docena, claveles) | X₄ ($/semana, ingreso familiar) | X₅ (tendencia) |
|-----------------|---------------------|----------------------|-------------------------|----------------------------------|----------------|
| 1971-III        | 11 484              | 2.26                 | 3.49                    | 158.11                           | 1              |
| 1971-IV         | 9 348               | 2.54                 | 2.85                    | 173.36                           | 2              |
| 1972-I          | 8 429               | 3.07                 | 4.06                    | 165.26                           | 3              |
| 1972-II         | 10 079              | 2.91                 | 3.64                    | 172.92                           | 4              |
| 1972-III        | 9 240               | 2.73                 | 3.64                    | 188.42                           | 5              |
| 1972-IV         | 8 862               | 2.77                 | 3.66                    | 198.62                           | 6              |
| 1973-I          | 6 216               | 3.59                 | 3.76                    | 186.28                           | 7              |
| 1973-II         | 8 263               | 2.60                 | 3.13                    | 180.49                           | 8              |
| 1973-III        | 7 315               | 2.89                 | 3.20                    | 183.33                           | 9              |
| 1973-IV         | 7 911               | 2.94                 | 3.12                    | 183.57                           | 10             |
| 1974-I          | 9 656               | 3.77                 | 3.65                    | 181.87                           | 11             |
| 1974-II         | 10 154              | 3.64                 | 3.60                    | 185.00                           | 12             |
| 1974-III        | 9 350               | 2.94                 | 3.25                    | 184.00                           | 13             |
| 1974-IV         | 8 886               | 2.94                 | 3.12                    | 188.20                           | 14             |
| 1975-I          | 7 718               | 4.24                 | 3.58                    | 175.67                           | 15             |
| 1975-II         | 5 872               | 3.69                 | 3.53                    | 188.00                           | 16             |

\* El autor agradece a Joe Walsh por recopilar estos datos de un mayorista en el área metropolitana de Detroit y por su subsecuente procesamiento.


In [14]:
df2 = pd.read_excel(T7_6_DIR , header=11)
df2

,YEAR,Y,X2,X3,X4,X5
0,1971.3,11484,2.26,3.49,158.11,1
1,1971.4,9348,2.54,2.85,173.36,2
2,1972.1,8429,3.07,4.06,165.26,3
3,1972.2,10079,2.91,3.64,172.92,4
4,1972.3,9240,2.73,3.21,178.46,5
5,1972.4,8862,2.77,3.66,198.62,6
6,1973.1,6216,3.59,3.76,186.28,7
7,1973.2,8253,3.23,3.49,188.98,8
8,1973.3,8038,2.60,3.13,180.49,9
9,1973.4,7476,2.89,3.20,183.33,10


# 3. Desembolsos del presupuesto de defensa de Estados Unidos, 1962-1981

| Año  | Desembolsos del presupuesto de defensa (Y₁) | PNB (X₁) | Ventas/Asistencias militares de Estados Unidos (X₂) | Ventas de la industria aeroespacial (X₃) | Conflictos 100,000+ (X₄) |
|------|--------------------------------------------|---------|-----------------------------------------------------|------------------------------------------|------------------------|
| 1962 | 51.1                                       | 560.3   | 0.6                                                 | 16                                       | 0                      |
| 1963 | 52.3                                       | 590.5   | 0.9                                                 | 16.4                                     | 0                      |
| 1964 | 53.6                                       | 632.4   | 1.1                                                 | 16.7                                     | 0                      |
| 1965 | 49.6                                       | 684.9   | 1.4                                                 | 17                                       | 1                      |
| 1966 | 56.8                                       | 749.9   | 1.6                                                 | 20.2                                     | 1                      |
| 1967 | 70.1                                       | 793.9   | 1.0                                                 | 23.4                                     | 1                      |
| 1968 | 80.5                                       | 865.0   | 0.8                                                 | 25.6                                     | 1                      |
| 1969 | 81.2                                       | 931.4   | 1.5                                                 | 24.6                                     | 1                      |
| 1970 | 80.3                                       | 992.7   | 1.0                                                 | 24.8                                     | 1                      |
| 1971 | 77.7                                       | 1077.6  | 1.5                                                 | 21.7                                     | 1                      |
| 1972 | 78.3                                       | 1185.9  | 2.95                                                | 21.5                                     | 1                      |
| 1973 | 74.5                                       | 1326.4  | 4.8                                                 | 24.3                                     | 0                      |
| 1974 | 77.8                                       | 1434.2  | 10.3                                                | 26.8                                     | 0                      |
| 1975 | 85.6                                       | 1549.2  | 16.0                                                | 29.5                                     | 0                      |
| 1976 | 89.4                                       | 1718.0  | 14.7                                                | 30.4                                     | 0                      |
| 1977 | 97.5                                       | 1918.3  | 8.3                                                 | 33.3                                     | 0                      |
| 1978 | 105.2                                      | 2163.9  | 11.0                                                | 38.0                                     | 0                      |
| 1979 | 117.7                                      | 2417.8  | 13.0                                                | 46.2                                     | 0                      |
| 1980 | 135.9                                      | 2633.1  | 15.3                                                | 57.6                                     | 0                      |
| 1981 | 162.1                                      | 2937.7  | 18.0                                                | 68.9                                     | 0                      |

Para explicar el presupuesto de defensa de Estados Unidos, considere el siguiente modelo:

- $ Y_t = \beta_1 + \beta_2 X_{2t} + \beta_3 X_{3t} + \beta_4 X_{4t} + \beta_5 X_{5t} + u_t $

Donde:

- $ Y_t = $ desembolsos del presupuesto de defensa durante el año t, $ miles de millones
- $ X_2t = $ PNB durante el año t, $ miles de millones
- $ X_3t = $ ventas militares de Estados Unidos/ayuda en el año t, $ miles de millones
- $ X_4t = $ ventas de la industria aeroespacial, $ miles de millones
- $ X_5t = $ conflictos militares que implican a más de 100,000 soldados. Esta variable adquiere el valor de 1 cuando participan 100,000 soldados o más, y es igual a cero cuando el número de soldados no llega a 100,000.

Para probar este modelo, se proporciona la información en la tabla 7.8 (archivo anexo).

### a) Estime los parámetros de este modelo y sus errores estándar, y obtenga $ R^2 $ y $ R^2 $ ajustada.

### b) Comente los resultados, considerando cualquier expectativa a priori que tenga sobre la relación entre $ Y $ y las diversas variables $ X $.

### c) ¿Qué otra(s) variable(s) incluiría en el modelo y por qué?

# 4. Gasto de consumo real, ingreso real, riqueza real y tasas de interés reales de Estados Unidos de 1947 a 2000.

En la tabla 7.12 (archivo anexo) se presenta la información de estas variables:

| Año  | C      | Yd      | Riqueza   | Tasa de Interés |
|------|--------|---------|-----------|-----------------|
| 1947 | 976.4  | 1035.2  | 5166.8    | -10.351         |
| 1948 | 998.1  | 1090.0  | 5280.8    | -4.720          |
| 1949 | 1025.3 | 1095.6  | 5607.4    | 1.044           |
| 1950 | 1090.9 | 1192.7  | 5759.5    | 0.407           |
| 1951 | 1107.1 | 1227.0  | 6086.1    | -5.283          |
| 1952 | 1142.4 | 1266.8  | 6243.9    | -0.277          |
| 1953 | 1197.2 | 1327.5  | 6355.6    | 0.561           |
| 1954 | 1221.9 | 1344.0  | 6797.0    | -0.138          |
| 1955 | 1310.4 | 1433.8  | 7172.2    | 0.262           |
| 1956 | 1348.8 | 1502.3  | 7375.2    | -0.736          |
| 1957 | 1381.8 | 1539.5  | 7315.3    | -0.261          |
| 1958 | 1393.0 | 1553.7  | 7870.0    | -0.575          |
| 1959 | 1470.7 | 1623.8  | 8188.1    | 2.296           |
| 1960 | 1510.8 | 1664.8  | 8351.8    | 1.511           |
| 1961 | 1541.2 | 1720.0  | 8971.9    | 1.296           |
| 1962 | 1617.3 | 1803.5  | 9091.5    | 1.396           |
| 1963 | 1684.0 | 1871.5  | 9436.1    | 2.058           |
| 1964 | 1784.8 | 2006.9  | 10003.4   | 2.027           |
| 1965 | 1897.6 | 2131.0  | 10562.8   | 2.112           |
| 1966 | 2006.1 | 2244.6  | 10522.0   | 2.020           |
| 1967 | 2066.2 | 2340.5  | 11312.1   | 1.213           |
| 1968 | 2184.2 | 2448.2  | 12145.4   | 1.055           |
| 1969 | 2264.8 | 2524.3  | 11672.3   | 1.732           |
| 1970 | 2317.5 | 2630.0  | 11650.0   | 1.166           |
| 1971 | 2405.2 | 2745.3  | 12312.9   | -0.712          |
| 1972 | 2550.5 | 2874.3  | 13499.9   | -0.156          |
| 1973 | 2675.9 | 3072.3  | 13081.0   | 1.414           |
| 1974 | 2653.7 | 3051.9  | 11868.8   | -1.043          |
| 1975 | 2710.9 | 3108.5  | 12634.4   | -3.534          |
| 1976 | 2868.9 | 3243.5  | 13456.8   | -0.657          |
| 1977 | 2992.1 | 3360.7  | 13786.3   | -1.190          |
| 1978 | 3124.7 | 3527.5  | 14450.5   | 0.113           |
| 1979 | 3203.2 | 3628.6  | 15340.0   | 1.704           |
| 1980 | 3193.0 | 3658.0  | 15965.0   | 2.298           |
| 1981 | 3236.0 | 3741.1  | 15965.0   | 4.704           |
| 1982 | 3275.5 | 3791.7  | 16312.5   | 4.449           |
| 1983 | 3454.3 | 3906.9  | 16944.8   | 4.691           |
| 1984 | 3640.6 | 4207.6  | 17526.7   | 5.848           |
| 1985 | 3820.9 | 4347.8  | 19068.3   | 4.331           |
| 1986 | 3981.2 | 4486.6  | 20530.0   | 3.768           |
| 1987 | 4113.4 | 4582.5  | 21235.7   | 2.819           |
| 1988 | 4279.5 | 4784.1  | 22332.0   | 3.287           |
| 1989 | 4393.7 | 4906.5  | 23659.8   | 4.318           |
| 1990 | 4474.5 | 5014.2  | 23105.1   | 3.595           |
| 1991 | 4466.6 | 5033.0  | 24050.2   | 1.803           |
| 1992 | 4594.5 | 5189.3  | 24418.2   | 1.007           |
| 1993 | 4748.9 | 5261.3  | 25092.3   | 0.625           |
| 1994 | 4928.1 | 5397.2  | 25218.6   | 2.206           |
| 1995 | 5075.6 | 5539.1  | 27439.7   | 3.333           |
| 1996 | 5237.5 | 5677.7  | 29448.2   | 3.083           |
| 1997 | 5423.9 | 5854.5  | 32664.1   | 3.120           |
| 1998 | 5683.7 | 6168.6  | 35587.0   | 3.584           |
| 1999 | 5968.4 | 6320.0  | 39591.3   | 3.245           |
| 2000 | 6257.8 | 6539.2  | 38167.7   | 3.576           |


### a) Con la información de la tabla, estime la función de consumo lineal usando los datos de ingreso, riqueza y tasa de interés. ¿Cuál es la ecuación ajustada?

### b) ¿Qué indican los coeficientes estimados sobre las relaciones entre las variables y el gasto de consumo?
